In [56]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
#retrieve links
import requests
import warnings
warnings.filterwarnings("ignore")
import time
from pathlib import Path

In [57]:
folder = Path("./raw")
files = (folder.glob("*.tfrecord"))

In [58]:
filenames=[]
for file in files:
    filenames.append(str(file))

In [59]:
filenames

['raw\\train5T.tfrecord',
 'raw\\trainbx.tfrecord',
 'raw\\trainDH.tfrecord',
 'raw\\traineP.tfrecord',
 'raw\\traingW.tfrecord',
 'raw\\trainlE.tfrecord',
 'raw\\trainmQ.tfrecord',
 'raw\\trainPJ.tfrecord',
 'raw\\trainve.tfrecord',
 'raw\\trainz2.tfrecord']

In [60]:
raw_dataset = tf.data.TFRecordDataset(filenames)

In [61]:
feature_description = {
    'id': tf.io.FixedLenFeature([], tf.string, default_value='')
}

In [62]:
def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)


parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

<MapDataset shapes: {id: ()}, types: {id: tf.string}>

In [63]:
lista=[]
for parsed_record in parsed_dataset.take(-1):
    lista.append(repr(parsed_record['id'])[-6:-2])

In [65]:
len(lista)

10123

In [66]:
skippati=[]

In [68]:
f=open('linksfin.txt','w')
for ID in tqdm(lista):
    url = ('https://data.yt8m.org/2/j/i/{}/{}.js').format(ID[:2],ID)
    session = requests.Session()
    session.trust_env = False
    headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
    
    try:
        check = session.get(url, verify=False, stream=True, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        skippati.append(url)
        continue
    #if check.status_code!=200:
    #    print(url, check.status_code)
 
    if check.status_code==403:
        skippati.append(url)
        continue
    a=(check.content)[1:-1]
    ending=(a.decode()).split(',')[1][1:-2]
    f.write(('https://www.youtube.com/watch?v={}\n').format(ending))
    #time.sleep(1)
f.close()
print(('#skipped videos: {}').format(len(skippati)))

100%|██████████████████████████████████████████████████████████████████████████| 10123/10123 [1:13:12<00:00,  2.30it/s]

#skipped videos: 1008
